In [7]:
import cellcharter as cc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import pandas as pd
import seaborn as sns
import scanpy as sc
import anndata as ad
import os
import json
from scipy.stats import mannwhitneyu, kruskal, ttest_ind
from statsmodels.stats.multitest import multipletests
import scikit_posthocs as sp
import warnings
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests
warnings.filterwarnings("ignore")

In [27]:
extension = 'svg'
save_path = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/paper/plots/figures/figure_6/{extension}/'
save_path_supp = f'/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/paper/plots/figures/suppl_figure_6/{extension}/'
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
adata = ad.read_h5ad("/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/results/standard/adatas/cells_final.h5ad")
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/neighborhood_color_map.json', 'r') as f:
    neighborhood_color_map = json.load(f)
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/phenotype_color_map.json', 'r') as f:
    phenotype_color_map = json.load(f)
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/disease_color_map.json', 'r') as f:
    disease_color_map = json.load(f)
with open('/Users/lukashat/Documents/PhD_Schapiro/Projects/Myeloma_Standal/github/myeloma_standal/src/Paper/figure_plots/disease3_color_map.json', 'r') as f:
    disease3_color_map = json.load(f)
neighborhood_colors = [neighborhood_color_map[cat] for cat in list(adata.obs['cellcharter_CN'].cat.categories)]
neighborhood_colors = ListedColormap(neighborhood_colors)
phenotype_colors = [phenotype_color_map[cat] for cat in list(adata.obs['Phenotype4'].cat.categories)]
phenotype_colors = ListedColormap(phenotype_colors)
disease_colors = [disease_color_map[cat] for cat in list(adata.obs['disease2'].cat.categories)]
disease_colors = ListedColormap(disease_colors)
disease3_colors = [disease3_color_map[cat] for cat in list(adata.obs['disease3'].cat.categories)]
disease3_colors = ListedColormap(disease3_colors)

In [ ]:
adata.obs['disease2'] = adata.obs['disease2'].astype(str)
adata = adata[adata.obs['disease2'].isin(['MM_BD', 'MM_noBD'])]
adata.obs['disease2'] = adata.obs['disease2'].astype('category')

In [29]:
celltype = 'PCs'
adata.obs['disease2'] = adata.obs['disease2'].astype(str)
adata = adata[adata.obs['disease2'].isin(['MM_BD', 'MM_noBD'])]
adata.obs['disease2'] = adata.obs['disease2'].astype('category')
disease_mapping = adata.obs[['disease2', 'image_ID']].drop_duplicates()
disease_mapping.set_index('image_ID', inplace=True)

In [34]:
df = adata[
    (adata.obs['Phenotype4'] == celltype)].to_df()
df['distance_to_bone_corrected'] = adata.obs.loc[df.index, 'distance_to_bone_corrected']
df['disease2'] = adata.obs.loc[df.index, 'disease2']
df['patient_ID'] = adata.obs.loc[df.index, 'patient_ID']
df['distance_bin'] = pd.cut(df['distance_to_bone_corrected'], 
                           bins=range(0, 601, 100))
df['distance_bin'] = df['distance_bin'].astype(str)

In [35]:
df = df[df['distance_bin'] != '(0, 100]']

In [36]:
df

,CD38,Perilipin,Vimentin,B4GALT1,MPO,CathepsinK,ATP5A,RUNX2,HIF1A,CD11b,...,CD3,CPT1A,CD98,HLA-DR,ST6GAL1,CD138,distance_to_bone_corrected,disease2,patient_ID,distance_bin
Object 159 in TS-373_IMC77_B_001.csv,0.627278,0.334125,0.795579,0.695620,0.683754,0.265274,0.802791,0.126574,0.392295,0.488467,...,0.348096,0.850838,0.473422,0.560415,0.604761,0.399426,187.416648,MM_BD,IMC77,"(100, 200]"
Object 275 in TS-373_IMC77_B_001.csv,0.598461,0.430989,0.862653,0.619337,0.639410,0.113938,0.600455,0.119701,0.466636,0.494048,...,0.305860,0.671503,0.483457,0.494721,0.518671,0.367571,192.634369,MM_BD,IMC77,"(100, 200]"
Object 302 in TS-373_IMC77_B_001.csv,0.614379,0.305964,0.659820,0.512564,0.610603,0.157633,0.664046,0.068907,0.439753,0.478042,...,0.299839,0.778511,0.451281,0.444483,0.542430,0.451115,199.379036,MM_BD,IMC77,"(100, 200]"
Object 323 in TS-373_IMC77_B_001.csv,0.648136,0.375997,0.376584,0.358691,0.354232,0.027791,0.206777,0.070086,0.442750,0.432926,...,0.479256,0.697037,0.551998,0.584196,0.459504,0.403593,193.313217,MM_BD,IMC77,"(100, 200]"
Object 334 in TS-373_IMC77_B_001.csv,0.755051,0.399660,0.422677,0.540883,0.497678,0.152986,0.230968,0.052511,0.331245,0.499366,...,0.485845,0.726174,0.523682,0.836143,0.427933,0.428162,187.779126,MM_BD,IMC77,"(100, 200]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Object 6566 in TS-373_IMC83_B_001.csv,0.765729,0.382974,0.763306,0.685703,0.328218,0.298350,0.835280,0.307949,0.623449,0.576319,...,0.492580,0.776737,0.594179,0.530810,0.561956,0.312485,186.453748,MM_BD,IMC83,"(100, 200]"
Object 6577 in TS-373_IMC83_B_001.csv,0.737180,0.368128,0.576205,0.613385,0.615378,0.292896,0.814401,0.193030,0.866189,0.651094,...,0.488338,0.758740,0.544445,0.462256,0.611176,0.364164,337.854998,MM_BD,IMC83,"(300, 400]"
Object 6592 in TS-373_IMC83_B_001.csv,0.726540,0.307771,0.507935,0.662648,0.277004,0.182987,0.779196,0.124444,0.637949,0.575766,...,0.492101,0.785394,0.618533,0.494449,0.628639,0.270694,148.734663,MM_BD,IMC83,"(100, 200]"
Object 6593 in TS-373_IMC83_B_001.csv,0.588351,0.300972,0.336872,0.676870,0.228257,0.202051,0.836995,0.137087,0.516838,0.546555,...,0.521410,0.831077,0.642627,0.506378,0.636885,0.119877,156.339374,MM_BD,IMC83,"(100, 200]"


In [39]:
?smf.mixedlm

Signature:
smf.mixedlm(
    formula,
    data,
    re_formula=None,
    vc_formula=None,
    subset=None,
    use_sparse=False,
    missing='none',
    *args,
    **kwargs,
)
Docstring:
Create a Model from a formula and dataframe.

Parameters
----------
formula : str or generic Formula object
    The formula specifying the model
data : array_like
    The data for the model. See Notes.
re_formula : str
    A one-sided formula defining the variance structure of the
    model.  The default gives a random intercept for each
    group.
vc_formula : dict-like
    Formulas describing variance components.  `vc_formula[vc]` is
    the formula for the component with variance parameter named
    `vc`.  The formula is processed into a matrix, and the columns
    of this matrix are linearly combined with independent random
    coefficients having mean zero and a common variance.
subset : array_like
    An array-like object of booleans, integers, or index
    values that indicate the subset of df to

In [38]:
model = smf.mixedlm('GLUT1 ~ disease2', df, groups=df['patient_ID'])
result = model.fit()
print(result.summary())

            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  GLUT1      
No. Observations:   266430   Method:              REML       
No. Groups:         65       Scale:               0.0268     
Min. group size:    256      Log-Likelihood:      104086.4032
Max. group size:    16097    Converged:           Yes        
Mean group size:    4098.9                                   
-------------------------------------------------------------
                    Coef. Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept           0.644    0.010 61.899 0.000  0.624  0.664
disease2[T.MM_noBD] 0.042    0.027  1.593 0.111 -0.010  0.094
Group Var           0.006    0.006                           



In [18]:
adata = adata[adata.obs['Phenotype4'] == 'PCs']
adata

View of AnnData object with n_obs × n_vars = 320147 × 33
    obs: 'Object', 'area', 'Y_centroid', 'X_centroid', 'axis_major_length', 'axis_minor_length', 'eccentricity', 'distance_to_bone', 'image_ID', 'disease', 'patient_ID', 'ROI', 'disease2', 'distance_to_bone_corrected', 'cellcharter_CN', 'Phenotype4', 'HistoneH3', 'disease3'
    var: 'name', 'channel', 'deepcell', 'mean', 'std'
    uns: 'Phenotype4_colors', 'cellcharter_CN_colors', 'spatial', 'spatial_neighbors'
    obsm: 'spatial'
    layers: 'arcsinh', 'zscore'
    obsp: 'spatial_connectivities', 'spatial_distances'

In [25]:
df = pd.DataFrame({'expression':adata[:, 'ATP5A'].X.toarray().flatten(),
                   'cohort': adata.obs['disease2'].values,
                   'patient': adata.obs['patient_ID'].values
                     })
df

,expression,cohort,patient
0,0.347032,MM_BD,IMC77
1,0.357526,MM_BD,IMC77
2,0.375071,MM_BD,IMC77
3,0.654439,MM_BD,IMC77
4,0.610767,MM_BD,IMC77
...,...,...,...
320142,0.649228,MM_BD,IMC83
320143,0.779196,MM_BD,IMC83
320144,0.836995,MM_BD,IMC83
320145,0.757393,MM_BD,IMC83


In [26]:
model = smf.mixedlm('expression ~ cohort', df, groups=df['patient'])
result = model.fit()
print(result.summary())

           Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: expression 
No. Observations:   320147  Method:             REML       
No. Groups:         65      Scale:              0.0147     
Min. group size:    264     Log-Likelihood:     220519.3223
Max. group size:    18527   Converged:          Yes        
Mean group size:    4925.3                                 
-----------------------------------------------------------
                  Coef. Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept         0.696    0.017 40.304 0.000  0.662  0.730
cohort[T.MM_noBD] 0.005    0.044  0.107 0.915 -0.082  0.091
Group Var         0.016    0.024                           

